In [32]:
from math import sqrt
import pandas as pd
import heapq

# Question 1

In [33]:
# Load in the Romania Map dataset 
file_path = './RomaniaMap.xlsx'
romania_map_data = pd.read_excel(file_path, sheet_name=None)

In [34]:
# Gets the node & edges data from the excel sheet 
nodes_df = romania_map_data['Nodes']
edges_df = romania_map_data['Edges']

# Extracts the coordinates & city names for each node
city_positions = {row['City']: (row['Lat (N)'], row['Long (E)']) for _, row in nodes_df.iterrows()}

In [35]:
# Heuristic Function to get the straight line distance
def heuristic(city1, city2):
    lat1, long1 = city_positions[city1]
    lat2, long2 = city_positions[city2]

    return sqrt((lat1 - lat2) ** 2 + (long1 - long2) ** 2)

In [36]:
# Main A* search algorithm to find the shortest path between 2 nodes/points
def astar_search(graph, start, goal):
    # Priority queue to store the cities to explore sorting them based on the lowest f = g + h score
    open_list = []
    heapq.heappush(open_list, (0, start))

    # Dictionary to keep track of the city that led to each city
    came_from = {start: None}

    # Stores the exact cost from the start to each city
    g_score =  {start: 0}

    while open_list:
        _, current_city =  heapq.heappop(open_list)

        if current_city == goal:
            path = []

            while current_city is not None:
                path.append(current_city)
                current_city = came_from[current_city]

            return path[::-1], g_score[goal]

        # If the current city = goal city, reconstruct the path by tracing back from goal to the start using the came_from dictionary and return both the path & total distance
        for neighbor, distance in graph[current_city]:
            tentative_g = g_score[current_city] + distance

            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score = tentative_g + heuristic(neighbor, goal)

                heapq.heappush(open_list, (f_score, neighbor))
                came_from[neighbor] = current_city
    
    return None, float('inf')

# Question 2

In [37]:
# Graph Representation using edges
graph = {}

for _, row in edges_df.iterrows():
    city_a, city_b, distance = row['Name(A)'], row['Name(B)'], row['Distance']
    graph.setdefault(city_a, []).append((city_b, distance))
    graph.setdefault(city_b, []).append((city_a, distance))

In [38]:
start_city = "Timisoara"
goal_city = "Bucharest"

conversion_rate = 0.621371
shortest_path, total_distance = astar_search(graph, start_city, goal_city)
total_distnace_in_miles = total_distance * conversion_rate

print(f"The shortest path from Timisora to Bucharest is {total_distnace_in_miles} miles")

The shortest path from Timisora to Bucharest is 333.05485600000003 miles


# Dicussion